In [ ]:
#primary requirements: # python3, numpy, pandas, xarray, dask, bottleneck, netCDF4, cftime, matplotlib
# run this script to check if all dependencies are installed and working
# also fixes np.NaN to np.nan in marineHeatWaves module

import sys, platform
import numpy as np
import pandas as pd, xarray as xr, dask, bottleneck, netCDF4, cftime, matplotlib
from pathlib import Path
from typing import List, Optional, Tuple, Dict, Union
import matplotlib.pyplot as plt
import os
import matplotlib.dates as mdates
from importlib.metadata import version, PackageNotFoundError
from glob import glob

# convert np.NAN to np.nan to avoid dependency issues
if not hasattr(np, "NaN"):
    np.NaN = np.nan  

print(sys.executable)
print(platform.python_version())

In [ ]:
# create daily mean files data from 3-hourly ERA5 wind data
from pathlib import Path
from glob import glob
import re
import pandas as pd
import xarray as xr
import dask

# set the file paths
# input 3-hourly files path
SRC_GLOB = "/home/Desktop/ERA5_wind_data_30S_30N_20E_110E/era5_wind_*.nc"
# output daily mean files path
OUT_DIR  = Path("/home/Desktop/Jupyter files/outputs/wind/daily")
OUT_DIR.mkdir(parents=True, exist_ok=True)

tname = "valid_time"     # file has 'valid_time' variable instead of 'time'
# variable name
u_name, v_name = "u10", "v10"

# set the data size and chunking for output files (adjust as needed)
ENC = {
    u_name: {"zlib": True, "complevel": 4, "dtype": "float32", "chunksizes": (90, 120, 120)},
    v_name: {"zlib": True, "complevel": 4, "dtype": "float32", "chunksizes": (90, 120, 120)},
}

dask.config.set(**{"array.slicing.split_large_chunks": True})

# open file files from path
def year_from_path(p: str):
    m = re.search(r"(19|20)\d{4}|(19|20)\d{2}", p)  
    """ Prefer pure 4-digit year; fallback if filenames have YYYYMM"""
    m4 = re.search(r"(19|20)\d{2}", p)
    return int(m4.group(0)) if m4 else None

def open_year(paths):
    """Keep each file as one contiguous chunk along time"""
    
    return xr.open_mfdataset(paths,combine="by_coords",engine="netcdf4",chunks={tname: -1},parallel=False,lock=False, decode_times=True,
                             drop_variables=None,)[[u_name, v_name]]

# collect files by year
paths = sorted(glob(SRC_GLOB))
by_year = {}
for p in paths:
    y = year_from_path(p)
    if y is not None:
        by_year.setdefault(y, []).append(p)

years = [y for y in sorted(by_year) if 1982 <= y <= 2024]
print(f"Found years: {years[0]}–{years[-1]} ({len(years)} years)")

# convert file per year 
for y in years:
    out_nc = OUT_DIR / f"era5_wind_daily_{y}.nc"
    if out_nc.exists():
        print(f"Skip file if {out_nc.name} already exists.")
        continue

    print(f"[{y}] opening {len(by_year[y])} file(s)…")
    ds3h = open_year(by_year[y])

    # daily mean file for having u10 and v10 variables
    daily = (ds3h.resample({tname: "1D"}).mean().astype("float32") .sortby(tname))

    # keep the file year same as input data
    daily = daily.sel({tname: slice(f"{y}-01-01", f"{y}-12-31")})

    # write NetCDF
    print(f"[{y}] writing {out_nc.name} …")
    daily.to_netcdf(out_nc, format="NETCDF4", engine="netcdf4", encoding=ENC)

    # print sample stats
    tt = pd.to_datetime(daily[tname].values)
    print(f"[{y}] saved {tt.size} daily steps; {tt[0].date()} → {tt[-1].date()}")

print("✓ Done. Daily files are saving in:", OUT_DIR)


In [ ]:
# open metadata and check the files structure and values after daily menaning and conversion to check if the files are correct and ready for analysis
import numpy as np, pandas as pd, xarray as xr
from glob import glob

FILES_GLOB = "/home/Desktop/ERA5 wind data/ERA5_daily_wind_data_30S_30N_20E_110E/era5_wind_daily_*.nc"

# open files path 
ds = xr.open_mfdataset(sorted(glob(FILES_GLOB)),combine="by_coords",engine="netcdf4",chunks={"valid_time": 1752})  

# Names
tname = "time" if "time" in ds.dims else "valid_time"
latn  = "lat" if "lat" in ds.coords else "latitude"
lonn  = "lon" if "lon" in ds.coords else "longitude"
u_name = "u10"; v_name = "v10"
print(f"Detected wind variables: u='{u_name}', v='{v_name}' (10 m)")

# coordinates & coverage
print("\n1st three cordinated ")
def _preview(arr):
    vals = arr.values
    if vals.ndim == 0:
        return f"{vals!r} (scalar)"
    return f"{vals[:3]} ... {vals[-3:]}"
for c in [tname, latn, lonn]:
    print(f"{c:>10s}: {_preview(ds[c])}")
if "expver" in ds.coords:
    print(f"{'expver':>10s}: {_preview(ds['expver'])}")

# print dimentions
print("\nDimensions:")
for k, v in ds.sizes.items():
    print(f"{k}: {v}")

# set the variable attributes & chunking data
def show_var(name):
    da = ds[name]
    print(f"\nVariable '{name}' attrs")
    for k, v in da.attrs.items():
        print(f"{k}: {v}")
    if hasattr(da.data, "chunks") and da.data.chunks:
        print("chunks:", da.data.chunks)

show_var(u_name)
show_var(v_name)

# quick check of time coverage & values 
t = pd.to_datetime(ds[tname].values)
t0, t1 = t[0], t[-1]
print(f"\n=== Time Coverage ===\nStart: {t0}  End: {t1}  Length: {t.size} steps")

# compute quick spatial means for the first and last day to check values
u, v = ds[u_name], ds[v_name]
first_day = slice(str(pd.to_datetime(t0).date()), str(pd.to_datetime(t0).date()))
last_day  = slice(str(pd.to_datetime(t1).date()), str(pd.to_datetime(t1).date()))

u_first = u.sel({tname: first_day}).mean([tname, latn, lonn]).compute().item()
v_first = v.sel({tname: first_day}).mean([tname, latn, lonn]).compute().item()
u_last  = u.sel({tname: last_day}).mean([tname, latn, lonn]).compute().item()
v_last  = v.sel({tname: last_day}).mean([tname, latn, lonn]).compute().item()
print(f"\nsample spatial velocities plotting u10 and v10 in m/s\n"
      f"{str(t0.date())}: u={u_first:.2f}, v={v_first:.2f}\n"
      f"{str(t1.date())}: u={u_last:.2f},  v={v_last:.2f}")


In [ ]:
# data preparation for zarr file computation and quick access for analysis of the daily mean wind data
import numpy as np, pandas as pd, xarray as xr
from pathlib import Path
from glob import glob

# file path for the daily mean wind data
FILES_GLOB = "/home/Desktop/ERA5 wind data/ERA5_daily_wind_data_30S_30N_20E_110E/era5_wind_daily_*.nc"

# North Indian Ocean ROI
ROI = dict(lat_min=0.0, lat_max=30.0, lon_min=40.0, lon_max=110.0)

# Quiver subsampling for readability, increase or decrease if arrows are too dense or less desnse
QUIVER_STEP = 6  

# open files path and load data
ds = xr.open_mfdataset(sorted(glob(FILES_GLOB)),combine="by_coords", engine="netcdf4",chunks={"valid_time": 365}  )

# find the standard lat, lon and valiriable names
tname = "time" if "time" in ds.dims else "valid_time"
latn  = "lat" if "lat" in ds.coords else "latitude"
lonn  = "lon" if "lon" in ds.coords else "longitude"
u_name, v_name = "u10", "v10"

# Slice ROI (handle descending latitude if present)
def slice_roi(ds, roi):
    lat = ds[latn]
    lon = ds[lonn]
    lat_slice = slice(roi["lat_min"], roi["lat_max"]) if lat[0] < lat[-1] else slice(roi["lat_max"], roi["lat_min"])
    lon_slice = slice(roi["lon_min"], roi["lon_max"]) if lon[0] < lon[-1] else slice(roi["lon_max"], roi["lon_min"])
    return ds.sel({latn: lat_slice, lonn: lon_slice})

ds_roi = slice_roi(ds[[u_name, v_name]], ROI)
ds_roi = ds_roi.chunk({tname: 365, latn: 80, lonn: 80})

print(ds_roi)


In [ ]:
# compute the daily zarr file for quick access and analysis of the daily mean wind data
from pathlib import Path

# Compute daily resultant magnitude of u10 and v10
ubar = ds_roi[u_name]
vbar = ds_roi[v_name]
speed = np.hypot(ubar, vbar)
daily_vec = xr.Dataset(
    data_vars=dict(ubar=ubar, vbar=vbar, speed=speed),
    coords={tname: ds_roi[tname], latn: ds_roi[latn], lonn: ds_roi[lonn]},
    attrs=dict(note="Daily mean 10m winds, ROI subset 0–30N, 40–110E; ubar/vbar are daily mean components; speed=hypot(ubar,vbar).")
)

# save zarr file path  
ROI_ZARR = "/home/Desktop/Jupyter files/outputs/wind/era5_u10_v10_daily_1982_2024_NIO_ROI.zarr"
# overwrite safely
if Path(ROI_ZARR).exists():
    import shutil, os
    shutil.rmtree(ROI_ZARR)
daily_vec.to_zarr(ROI_ZARR, mode="w")
print(f"Saved zarr file to:\n  {ROI_ZARR}\n", daily_vec)



In [ ]:
# wind data plot using matlplotlib boraderline

import shapefile  # pyshp
import numpy as np
import matplotlib.pyplot as plt

COAST_SHP = "/home/Desktop/cartopy_data/shapefiles/natural_earth/physical/110m/ne_110m_coastline.shp"

def _roi_bbox(roi):
    # Return (minx, miny, maxx, maxy) in lon/lat
    return (roi["lon_min"], roi["lat_min"], roi["lon_max"], roi["lat_max"])

def draw_coastline(ax, roi, lw=0.8, color="black", alpha=1.0):
    
    r = shapefile.Reader(COAST_SHP)
    minx, miny, maxx, maxy = _roi_bbox(roi)

    for shape_rec in r.shapeRecords():
        shp = shape_rec.shape
        # Quick reject using the shape bbox
        sxmin, symin, sxmax, symax = shp.bbox
        if (sxmax < minx) or (sxmin > maxx) or (symax < miny) or (symin > maxy):
            continue
        # Each shapefile "part" is a continuous polyline
        pts = np.asarray(shp.points)
        parts = list(shp.parts) + [len(pts)]
        for i0, i1 in zip(parts[:-1], parts[1:]):
            seg = pts[i0:i1]
            # Optional: clip crudely to ROI to avoid long segments
            mask = (
                (seg[:,0] >= minx-1) & (seg[:,0] <= maxx+1) &
                (seg[:,1] >= miny-1) & (seg[:,1] <= maxy+1)
            )
            if mask.any():
                ax.plot(seg[mask,0], seg[mask,1], color=color, lw=lw, alpha=alpha)

    # Also draw a neat rectangular ROI frame (ocean/land border feel)
    ax.plot([minx, maxx, maxx, minx, minx],
            [miny, miny, maxy, maxy, miny],
            color="black", lw=0.8)



# (Re)open the compact ROI Zarr if needed
vec = xr.open_zarr(ROI_ZARR, chunks={"valid_time": 365, latn: 80, lonn: 80})

# Long-term mean vector (arrow direction only)
U = vec["ubar"].mean(tname)
V = vec["vbar"].mean(tname)

# Subsample for quiver clarity
Uq = U.isel({latn: slice(None, None, QUIVER_STEP), lonn: slice(None, None, QUIVER_STEP)})
Vq = V.isel({latn: slice(None, None, QUIVER_STEP), lonn: slice(None, None, QUIVER_STEP)})

# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.set_title("North Indian Ocean — Long-term Mean Wind (1982–2024)")

# white background + ROI border as black box
lon = U[lonn].values
lat = U[latn].values
ax.set_xlim(lon.min(), lon.max()); ax.set_ylim(lat.min(), lat.max())
for spine in ax.spines.values():
    spine.set_edgecolor("black"); spine.set_linewidth(0.9)

# Quiver
ax.quiver(Uq[lonn], Uq[latn], Uq, Vq, scale=120, width=0.002)
draw_coastline(ax, ROI, lw=0.8, color="green", alpha=1.0)

ax.set_xlabel("Longitude"); ax.set_ylabel("Latitude")
ax.grid(True, ls="--", lw=0.4, alpha=0.4)
plt.tight_layout(); plt.show()